## Conceptual introduction

Recommender systems often involve applying some kind of an algorithm to the so called **user-item ratings matrix** of size $NxM$ where $N$ - number of users, $M$ - number of items. This matrix is sparse which is good since our goal is to recommend products to users and it would be nice to have something to recommend in the first place.

The value in a cell could be a rating (eg. on a scale from 1 to 5 - star-based system), whether a user rated an item positively or negatively (thumb-up / thumb-down approach). This type of information is called **explicit** - we know the real attitude of a particular user toward an item. The second type of information used in recommender systems is **implicit** rating - we only know that a user browsed an item, watched a movie, listened to a song etc. - in that case we can't tell if he/she liked it explicitly, we can only assume that. It is often concluded that the more times a user interacted with an item/song/movie the more confidence we have that he/she in fact likes it.

The implementation presented below assumes we only deal with explicit ratings.

| UserId \ ItemId | Item 1 | Item 2 | Item 3 | Item 4 | ... | Item M |
|:---------------:|--------|--------|--------|--------|-----|--------|
| **User 1**          |    2   |    4   |    4   |    3   | ... |   4.5  |
| **User 2**          |   3.5  |    5   |   4.5  |   2.5  | ... |    4   |
| **User 3**          |    1   |    2   |    5   |   1.5  | ... |    5   |
| **...**             |   ...  |   ...  |   ...  |   ...  | ... |   ...  |
| **User N**          |    4   |   3.5  |    4   |   4.5  | ... |   2.5  |

<div style="text-align: center">Example of user-item ratings matrix</div>


In [4]:
import numpy as np
import pandas as pd
import random
from itertools import product
from datetime import datetime

# Data

As the main goal of this notebook is to explain the intuition behind the most common algorithms related to recommender systems and not efficiency of implementation, the dataset size is small. It is based on well-known movielens dataset available [here](https://grouplens.org/datasets/movielens/). The dataset was filtered to include only the most popular movies (top 1000) and the most active users (5000 users). Thus the dimensionality of user-item matrix is $5000x1000$ but the algorithm would work equally on larger dataset as well (at the cost of longer training time).

In [ ]:
all_ratings = pd.read_csv('/home/kuba/Desktop/colab_filt_data/rating.csv', usecols=[0,1,2])

# calculate 1000 most popular movies
mov_chosen = all_ratings.groupby('movieId')['userId'].nunique().sort_values().iloc[-1000:].index.values

# calculate top 5000 users who rated the most movies
user_chosen = all_ratings.groupby('userId')['movieId'].nunique().sort_values().iloc[-5000:].index.values

# filter a data frame to selected movies and users
# we need to call copy() method, because selecting rows from dataframe only creates a view of original df
# hence we have to make a new df by calling copy(). It lets us overwrite the indexes
ratings_filt = all_ratings[all_ratings.userId.isin(user_chosen) & all_ratings.movieId.isin(mov_chosen)].copy()
del all_ratings

# save the filtered data frame
ratings_filt.to_csv('C/home/kuba/Desktop/colab_filt_data/ratings_filtered.csv', index=False)

## Collaborative filtering

Collaborative filtering aims to find a set of similar users for each of the users (by calculating correlation of ratings for each pair of users)